> Copyright 2022 Bruno César Brito Sant'Anna <mail@bsantanna.me>
>
> Licensed under the Apache License, Version 2.0 (the "License");
> you may not use this file except in compliance with the License.
> You may obtain a copy of the License at
>
>     http://www.apache.org/licenses/LICENSE-2.0
>
> Unless required by applicable law or agreed to in writing, software
> distributed under the License is distributed on an "AS IS" BASIS,
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
> See the License for the specific language governing permissions and
> limitations under the License.

In [1]:
import json
import os
import re

import pandas as pd

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf

In [2]:
# Load processed iban dataframe
iban_data = pd.read_csv("../processed_ibans.csv", sep=";")

In [3]:
# Load json trained model
json_model = tf.keras.models.load_model("../training/json/bin")

In [4]:
# Create a simple Validator callable
class Mod97Validator:

    def __init__(self):
        self.mod_97_alphabet = {
            "A": 10, "B": 11, "C": 12, "D": 13,
            "E": 14, "F": 15, "G": 16, "H": 17,
            "I": 18, "J": 19, "K": 20, "L": 21,
            "M": 22, "N": 23, "O": 24, "P": 25,
            "Q": 26, "R": 27, "S": 28, "T": 29,
            "U": 30, "V": 31, "W": 32, "X": 33,
            "Y": 34, "Z": 35,
        }

    def __call__(self, iban):
        mod_97_input = (iban[4:] + iban[:4]).upper()
        for letter, number in self.mod_97_alphabet.items():
            mod_97_input = mod_97_input.replace(letter, str(number))
        return int(mod_97_input) % 97 == 1


# initialize simple model
mod97_validator = Mod97Validator()

In [5]:
# Utility functions
def validate(iban):
    # get input parameters
    bban = iban[4:]
    code = iban[:2].upper()
    size = len(iban)
    check_digit = iban[2:4]

    # get encoded and json predictions
    encoded_prediction = json_model.predict(
        {"Code": tf.convert_to_tensor([code]), "Size": tf.convert_to_tensor([size])}, verbose=0
    )
    json_prediction = iban_data["JSON"][encoded_prediction.argmax()]

    # get validation dictionary from predicted json
    validation_dict = json.loads(json_prediction)

    # validate country code
    if validation_dict["code"] != code:
        return False

    # validate size
    if validation_dict["size"] != len(iban):
        return False

    # validate check digit
    if not re.match(validation_dict["check_digit_regex"], check_digit):
        return False

    # validate bban
    if not re.match(validation_dict["bban_regex"], bban):
        return False

    # validate mod 97¸
    return mod97_validator(iban)


In [6]:
# Example from https://bank.codes/iban/examples/
input = "BR1800000000141455123924100C2"
is_valid = validate(input)
print("Input IBAN: {}, valid: {}".format(input, is_valid))

Input IBAN: BR1800000000141455123924100C2, valid: True
